In [89]:
# Install necessary packages
!pip install --upgrade label-studio-sdk 
!pip install docling

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pylatexenc-2.10.tar.gz (162 kB)
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.3-py3-none-macosx_10_9_universal2.whl.metadata (5.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 39.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 30.0 MB/s eta 0:00:00a 0:00:01
Using cached ninja-1.11.1.3-py3-none-macosx_10_9_universal2.whl (279 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 40.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136863 sha256=663e740d60304fde465488dc05999e8b84398ef8cdb4340ec33775bca05c382a
  Stored in directory: /Users/mkaplan/Library/Caches/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [163]:
### API Keys and URLS 

# URL of your Label Studio Instance
label_studio_url = "http://localhost:8080" 
# Label Studio API Key (in your user settings)
label_studio_api_key = 


## Create Label Studio Project for OCR

In [164]:
from label_studio_sdk.client import LabelStudio

# Connect to Label Studio 
ls = LabelStudio(
    base_url=label_studio_url,  
    api_key=label_studio_api_key,
)

In [165]:
# Labeling Config for OCR using Multi-page document annotation
# We draw bounding boxes over the text in the image, and then put the text inside the bounding box

labeling_config = """
<View>
  <Image name="image" valueList="$pages"/>
  <Labels name="label" toName="image">
    <Label value="Text" background="green"/>
  </Labels>
  <Rectangle name="bbox" toName="image" strokeWidth="3" perRegion="true"/>
  <TextArea name="transcription" toName="image"
            editable="true"
            perRegion="true"
            required="true"
            maxSubmissions="1"
            rows="5"
            placeholder="Recognized Text"
            displayMode="region-list"
            />
</View>
"""

In [166]:
# Create Label Studio Project
project = ls.projects.create(
    title="SmolDocling",
    description="OCR using the new SmolDocling model",
    label_config=labeling_config
)


## Set up sample task 

In [167]:
# This is a single sample task: a 3 page document. If you want to replace with your own task, replace the pages list 
# with a list of URLs to your document in iamge form. 
# You can make this a loop to add multiple documents! 

sample_task = {
    "pages": [
      "https://htx-pub.s3.amazonaws.com/demo/images/demo_stock_purchase_agreement/0001.jpg",
      "https://htx-pub.s3.amazonaws.com/demo/images/demo_stock_purchase_agreement/0002.jpg",
      "https://htx-pub.s3.amazonaws.com/demo/images/demo_stock_purchase_agreement/0003.jpg"
    ]
}

In [168]:
# Upload task to Label Studio 
task = ls.tasks.create(
    project=project.id,
    data=sample_task
)

## Do OCR with SmolDocling

In [177]:
# Helper Function 

def convert_bbox_to_ls(bbox, width, height): 
    """
    BBoxes in SmolDocling are given in LTRB format. We need to convert to the format Label Studio expects:
    the top left coordinate as a percentage of the total image, and the width and height as percents. 

    Args: 
        bbox: the bbox dictionary from SmolDocling's response object
        width: the width of the image in pixles 
        height: the height of the image in pixles 

    Returns: a dictionary containing all the information for the value field in Label Studio for Rectangle Labels.
    """
    #convert bbox to LS format
    bbox_x = (bbox["l"] / width) * 100 
    bbox_y = 100 - ((bbox["t"] / height) * 100)
    bbox_width = ((bbox["r"] - bbox["l"]) / width) * 100
    bbox_height = ((bbox["t"]- bbox["b"]) / height) * 100

    return {"x": bbox_x, 
            "y": bbox_y, 
            "width" : bbox_width, 
            "height" : bbox_height, 
           "rectanglelabels": ["Text"]}


In [178]:
# Helper Function
from PIL import Image
import requests
from io import BytesIO

def load_image_from_url(url):
    """Loads an image from a URL using PIL.

    Args:
        url: The URL of the image.

    Returns:
        An Image object if successful, None otherwise.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        image = Image.open(BytesIO(response.content))
        return image
    except requests.exceptions.RequestException as e:
        print(f"Error fetching image from {url}: {e}")
        return None
    except Exception as e:
         print(f"Error opening image from {url}: {e}")
         return None

In [184]:
def do_ocr(task_pages):
    """
    Do the OCR on the list of images. 

    Args: 
        task_pages: a list of the urls for each image in the task

    Output: 
    
    """
    predictions = []
    for idx, page in enumerate(task_pages):
        
        # Load the image as PIL to get the original height and width
        image = load_image_from_url(page)
        width, height = image.size

        # Convert from image to SmolDocling document 
        converter = DocumentConverter()
        result = converter.convert(page)
        output_json = result.document.export_to_dict()

        # Process output josn
        for recognized_text in output_json["texts"]:
            bbox = recognized_text["prov"][0]["bbox"]
            bbox_dict = convert_bbox_to_ls(bbox, width, height)
            
            text = recognized_text["text"]
            pred = {"item_index": idx, "bbox_value" : bbox_dict, "text_value": text}
            print(pred)
            predictions.append(pred)
            
    return predictions 
            
        
        

## Create SmolDocling Predictions and Upload to Label Studio

In [183]:
# main method 
import time
import os
from PIL import Image
from docling.document_converter import DocumentConverter


# Get project information for uploading predictions 
upload_project = ls.projects.get(id=project.id)
li = upload_project.get_label_interface()


# for every task in the project, we'll get the SmolDocling predictions and upload them
for task in ls.tasks.list(project=upload_project.id):
    task_id = int(task.id)
    print(task_id)
    task_pages = task.data["pages"]
    task_predictions = do_ocr(task_pages)
    print("OCR Completed")
    print("Creating Predictions for Label Studio")
    results = [
        {
            "id" : f"region{i}", 
            "from_name" : "label",
            "to_name" : "image",
            "type": "rectanglelabels",
            "value" : p["bbox_value"],
            "item_index" : p["item_index"],

        }
    for i, p in enumerate(task_predictions)]
    results.extend([{
        "id": f"region{i}", 
        "from_name" : "transcription", 
        "to_name" : "image", 
        "type" : "textarea", 
        "value" : {
            "text" : [p["text_value"]]
        },
        "item_index": p["item_index"]
    } for i, p in enumerate(task_predictions)])
        
    ls.predictions.create(task=task_id, result=results, model_version="SmolDocling")
    print(f"prediction for task {task_id} uploaded")

27
{'item_index': 0, 'bbox_value': {'x': 22.958333333333332, 'y': 9.24315619967794, 'width': 53.91666666666667, 'height': 1.6425120772946862, 'rectanglelabels': ['Text']}, 'text_value': 'SERIES B PREFERRED STOCK PURCHASE AGREEMENTT'}
{'item_index': 0, 'bbox_value': {'x': 11.458333333333334, 'y': 12.43156199677938, 'width': 77.08333333333334, 'height': 7.342995169082125, 'rectanglelabels': ['Text']}, 'text_value': 'THIS SERIES B PREFERRED STOCK PURCHASE AGREEMENT (this Delaware corporation (the "Company\' and the investors listed on Exhibit A attached to this Agreement (each a "Purchaser and together the "Purchasers\''}
{'item_index': 0, 'bbox_value': {'x': 17.458333333333332, 'y': 20.83735909822866, 'width': 28.833333333333332, 'height': 2.0933977455716657, 'rectanglelabels': ['Text']}, 'text_value': 'The partics hereby agree as follows:'}
{'item_index': 0, 'bbox_value': {'x': 17.5, 'y': 24.219001610305952, 'width': 35.5, 'height': 1.6747181964573306, 'rectanglelabels': ['Text']}, 'tex